<a href="https://colab.research.google.com/github/Subhasmitamishra88/BIG-DATA-SPARK-NOSQL/blob/main/BDSN_Assignment_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update > /dev/null
!apt install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Get latest and correct version of Spark
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#
#!wget -q http://apache.osuosl.org/spark/spark-2.2.2/spark-2.2.2-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark
!pip install -q pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [ ]:
from pyspark.sql import SparkSession
#spark = SparkSession.builder.master("local[*]").getOrCreate()
# note UI port switched from default 4040 to 4050 to avoid clash with ngrok
spark = SparkSession.builder.master("local[*]").config('spark.ui.port', '4050').getOrCreate()
sc = spark.sparkContext

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MLlibLRProject').getOrCreate()

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving cruise_ship_info.csv to cruise_ship_info (1).csv


In [ ]:
data = spark.read.csv('cruise_ship_info.csv',header=True, inferSchema=True)

In [ ]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder

In [ ]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_lineIndex")
indexed = indexer.fit(data).transform(data)
indexed.head(2)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.277, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_lineIndex=1.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.277, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_lineIndex=1.0)]

In [ ]:
onehotencoder = OneHotEncoder(inputCol="Cruise_lineIndex",
                        outputCol="Cruise_lineVec")
data_encoded = onehotencoder.fit(indexed).transform(indexed)
data_encoded.head(2)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.277, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_lineIndex=1.0, Cruise_lineVec=SparseVector(1, {})),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.277, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_lineIndex=1.0, Cruise_lineVec=SparseVector(1, {}))]

In [ ]:
data_encoded.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_lineIndex: double (nullable = false)
 |-- Cruise_lineVec: vector (nullable = true)



In [ ]:
# VectorAssembler to bring the data to the MLlib format
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Age','Tonnage','passengers','length','cabins','passenger_density',
                                       'Cruise_lineVec'],
                            outputCol = 'features')
final_data = assembler.transform(data_encoded)
final_data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_lineIndex: double (nullable = false)
 |-- Cruise_lineVec: vector (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
#train test split
train_data , test_data = final_data.randomSplit([0.7,0.3])

In [ ]:
# Linear regression model
lr = LinearRegression()

In [ ]:
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

In [ ]:
regEvaluator = RegressionEvaluator()
cv = CrossValidator(estimator=lr,
                           estimatorParamMaps=paramGrid,
                           evaluator=regEvaluator,
                           numFolds=2)
cvmodel = cv.fit(train_data.withColumnRenamed('crew','label').select('label','features'))


In [ ]:
test_results = cvmodel.transform(test_data.withColumnRenamed('crew','label').select('label','features'))

In [ ]:
test_results.show()

+-----+--------------------+-----------------+
|label|            features|       prediction|
+-----+--------------------+-----------------+
| 10.0|[17.0,101.353,26....|19.02214292407582|
|  9.2|[22.0,70.367,20.5...|9.319751311121749|
|  9.2|[15.0,70.367,20.5...|9.157672112556845|
+-----+--------------------+-----------------+



In [ ]:
regEvaluator.setMetricName('rmse') # or set metricName="rmse" when defined regEvaluator

RegressionEvaluator_d74ac04f39a5

In [ ]:
regEvaluator.evaluate(cvmodel.transform(test_data.withColumnRenamed('crew','label').select('label','features')))

5.20945278537078